In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import gpflow
import tensorflow as tf
import time

from gpflow.ci_utils import ci_niter

In [3]:
scaled = pd.read_csv('../data/scaled_20Kcells.csv', index_col=0)
pseudotime = pd.read_csv('../data/pseudotime_20Kcells.csv', index_col=0)

In [4]:
# select cells from a lineage
np.random.seed(1)
num_samples = 1000
num_genes = 100

curve_id = 1
cells = pseudotime[~pseudotime['curve{}'.format(curve_id)].isna()].index
cells = np.random.choice(cells, num_samples)
genes = np.random.choice(scaled.columns, num_genes)

data = scaled.loc[cells, genes]
times = pseudotime.loc[cells, 'curve{}'.format(curve_id)]
times = times.rename('time')
Y = data.values

In [13]:
%%time
# model to learn 1D pseudotime

latent_dim = 2  # number of latent dimensions
num_inducing = 20  # number of inducing pts
num_data = Y.shape[0]  # number of data points

np.random.seed(42)  # fix random initialization

X_mean_init = gpflow.utilities.ops.pca_reduce(Y, latent_dim)
X_var_init = np.ones((num_data, latent_dim))
Z = np.random.permutation(X_mean_init)[:num_inducing] #np.linspace(np.min(X_mean_init), np.max(X_mean_init), num_inducing)[:, None] 

kernel = gpflow.kernels.SquaredExponential(lengthscales=[1.0] * latent_dim)

bgplvm = gpflow.models.BayesianGPLVM(
    Y,
    X_data_mean=X_mean_init,
    X_data_var=X_var_init,
    kernel=kernel,
    inducing_variable=Z
)

CPU times: user 22 ms, sys: 24.4 ms, total: 46.4 ms
Wall time: 41.4 ms


#### Scipy optimizer efficiency
* On the same set of data (Y = [1000, 10], M = 20, Q = 2)
    * gpflow2 took ~3min (Wall), ~5min (CPU) 
    * gpflow1 took 4s (Wall), 5s (CPU).
* On full data, 
    * gpflow2 hangs on 1 iteration for at least half an hour (or something close)
    * gpflow1 finishes in 8s (Wall), 11s (CPU)

In [12]:
%%time
opt = gpflow.optimizers.Scipy()
maxiter = ci_niter(50)
_ = opt.minimize(
    bgplvm.training_loss,
    method="BFGS",
    variables=bgplvm.trainable_variables,
    options=dict(maxiter=maxiter, disp=True),
)

         Current function value: 101408.319329
         Iterations: 50
         Function evaluations: 62
         Gradient evaluations: 62
CPU times: user 4min 59s, sys: 8.37 s, total: 5min 7s
Wall time: 2min 41s


In [22]:
%%time
@tf.function
def optimization_step(model: gpflow.models.BayesianGPLVM):
    adam_opt.minimize(model.training_loss, var_list=model.trainable_variables)
    elbo = model.elbo()
    return elbo

adam_opt = tf.optimizers.Adam(learning_rate=0.01)

logf = []
tol = 1e-4
log_freq = 1
num_iterations = 50

tf.print('initial elbo {:.4f}'.format(m.elbo()))

for step in range(num_iterations):
    start_time = time.time()
    elbo = optimization_step(m)
    logf.append(elbo)
    if step > 0 and np.abs(elbo - logf[-2]) < tol:
        tf.print('converge at iteration {} elbo {:.4f}'.format(step+1, elbo))
        break
    if (step + 1)  % log_freq == 0:
        tf.print('iteration {} elbo {:.4f}, took {:.4f}s'.format(step+1, elbo, time.time()-start_time))

initial elbo -44681372.6975
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function optimization_step at 0x14c5c2d08>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function optimization_step at 0x14c5c2d08>. Note that functions defined in certain environments,

KeyboardInterrupt: 

In [28]:
curve_id = 1
L = 2
with open('../outputs/gplvm_L{}_curve{}.pkl'.format(L, curve_id), 'rb') as f:
    output = pickle.load(f)